This file will cover the entire data pipeline process for acquiring all the data, cleaning, and processing for the sake of this project

# Importing and configurations, also folder setup

In [42]:
import os # Note: for working with folders / paths
from datetime import datetime

import pandas as pd
import yfinance as yf             # Note: To download stock/market data
from fredapi import Fred          # Note: To download macroeconomic data from FRED
from pytrends.request import TrendReq  # Note: To pull Google Trends sentiment data
import duckdb                     # Note: This one for now is unsure-ish, can be used for storing/querying data with SQL

# Note: This is the BASE_DIR = the folder where this notebook is running
BASE_DIR = os.getcwd()

# # Note: This will be the main data directory:
DATA_DIR = os.path.join(BASE_DIR, "data")

# Note: Subfolders for different stages of the pipeline:
RAW_DIR = os.path.join(DATA_DIR, "raw")
CLEANED_DIR = os.path.join(DATA_DIR, "cleaned")
MERGED_DIR = os.path.join(DATA_DIR, "merged")

# Note: Now also creating more specific subfolders for raw data, each housing different stuff:
# Note: data/raw/prices/     -> raw price data (S&P 500, VIX, etc.)
# Note: data/raw/macro/      -> raw macro data (CPI, unemployment, etc.)
# Note: data/raw/sentiment/  -> raw sentiment data (Google Trends, etc.)
RAW_PRICES_DIR = os.path.join(RAW_DIR, "prices")
RAW_MACRO_DIR = os.path.join(RAW_DIR, "macro")
RAW_SENTIMENT_DIR = os.path.join(RAW_DIR, "sentiment")

# Note: Same structure for cleaned data:
# Note: data/cleaned/prices/
# Note: data/cleaned/macro/
# Note: data/cleaned/sentiment/
CLEAN_PRICES_DIR = os.path.join(CLEANED_DIR, "prices")
CLEAN_MACRO_DIR = os.path.join(CLEANED_DIR, "macro")
CLEAN_SENTIMENT_DIR = os.path.join(CLEANED_DIR, "sentiment")

# Note: We want historical data starting from 1980, so setting that as a golbal variable
START_DATE = "1980-01-01"

# Note: Today's date (so we always pull up-to-date data)
TODAY = datetime.today().strftime("%Y-%m-%d")

# Note: These will be the financial tickers (for yfinance)
# Note: ^GSPC = S&P 500 index; ^VIX  = Volatility Index
TICKERS = {
    "sp500": "^GSPC",
    "vix": "^VIX",
}

# Note: Next we have the FRED macroeconomic series IDs
# Note: These IDs come from the FRED website
FRED_SERIES = {
    "cpi": "CPIAUCSL",           # Note: Consumer Price Index
    "unemployment": "UNRATE",    # Note: Unemployment rate
    "fed_funds_rate": "FEDFUNDS",
    "m2_money_stock": "M2SL",
}

# Note: Finally the Google Trends sentiment settings
# Note: These are some of the keywords we care about for sentiment around crashes/recessions.
TRENDS_KEYWORDS = [
    # "recession",
    # "stock market crash",
    "bear market",
    "financial crisis",
    "economic crisis",
    "market crash",
    "economic collapse",
    "market panic",
    "market volatility",
    "stock crash",
    "credit crunch",
    "bank run",
    "bank collapse",
    "investor panic",
    "yield curve inversion",
    "inflation crisis",
    "stagflation",
    "should I sell my stocks",
    "why is the stock market dropping",
    "market uncertainty",
    "fed meeting",
    "inflation report",
    "jobs report"
]
# Note: Google Trends only goes back to 2004, so our sentiment data starts then.
TRENDS_START_DATE = "2004-01-01"

# Note: Let's print it so we can make sure most of this configuration and folder setup worked
print("Base directory:", BASE_DIR)
print("Data directory:", DATA_DIR)
print("Raw data folder:", RAW_DIR)
print("Cleaned data folder:", CLEANED_DIR)
print("Merged data folder:", MERGED_DIR)


Base directory: /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code
Data directory: /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data
Raw data folder: /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/raw
Cleaned data folder: /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/cleaned
Merged data folder: /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/merged


# Acquire price data (S&P 500 + VIX) by downloading all of it

In [43]:
# Note: This is where we will download the actual price data using yfinance, this includes the S&P 500 (^GSPC) and the VIX index (^VIX)
# Note: We'll pull all data from our START_DATE (1980) up through TODAY
# Note: The print statements below are practices we are adopting so as to make sure at each step we understand what is causing an error if something happens for debugging

def download_prices(name, ticker, start=START_DATE, end=TODAY):
    # Note: This function downloads price data for a given ticker.
    # Note: 'name' is just our friendly label like "sp500".
    # Note: 'ticker' is the actual Yahoo Finance symbol like "^GSPC".

    print(f"[prices] Downloading {name} ({ticker}) from {start} to {end} ...")

    # Note: yf.download() pulls OHLCV data (Open, High, Low, Close, Volume)
    df = yf.download(ticker, start=start, end=end)

    # Note: If somehow yfinance returns nothing, we don't want the code to break.
    if df.empty:
        print(f"[prices] WARNING: no data returned for {ticker}")
        return None
    
    # Note: So first gotta convert all column names into plain strings because one of the column names in yfinance seem to not be str but a tuple
    df.columns = df.columns.map(str)

    # Note: Moving the index into a normal 'date' column and cleaning column names a bit.
    df = df.reset_index()
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]

    return df


def save_prices(df, name):
    # Note: This function simply saves the dataframe to the correct folder.
    # Note: If df is None or empty, then there's nothing to save.

    if df is None or df.empty:
        print(f"[prices] Nothing to save for {name}")
        return

    # Note: Construct the path like: data/raw/prices/sp500_prices_raw.csv
    out_path = os.path.join(RAW_PRICES_DIR, f"{name}_prices_raw.csv")

    # Note: Saving all of the data
    df.to_csv(out_path, index=False)

    print(f"[prices] Saved {name} data to {out_path} (rows: {len(df)})")


# Note: Now we loop through all tickers we defined earlier (S&P 500 + VIX)
# Note: For each ticker, we download it and then save it.
for name, ticker in TICKERS.items():
    df = download_prices(name, ticker)
    save_prices(df, name)


/var/folders/bv/q8ls960x2513d35yd59cvhc00000gn/T/ipykernel_54145/2921958070.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/var/folders/bv/q8ls960x2513d35yd59cvhc00000gn/T/ipykernel_54145/2921958070.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed

[prices] Downloading sp500 (^GSPC) from 1980-01-01 to 2025-11-12 ...
[prices] Saved sp500 data to /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/raw/prices/sp500_prices_raw.csv (rows: 11561)
[prices] Downloading vix (^VIX) from 1980-01-01 to 2025-11-12 ...
[prices] Saved vix data to /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/raw/prices/vix_prices_raw.csv (rows: 9033)


# Get the macro data from FRED

In [44]:
# Note: For loading the .env variables like API keys
from dotenv import load_dotenv
load_dotenv()

# Note: Already went to the FRED website and got an API key, which is in the .env (so git won't track it), so will use that for this step
# Note: Now we move on to downloading macroeconomic data from FRED
# Note: This will give us things like CPI, unemployment, Fed funds rate, and M2 money stock

# Note: Just pulling that API key from the .env
FRED_API_KEY = os.getenv("FRED_API_KEY")



# Note: Now we will create the FRED client using the key
fred = None
try:
    fred = Fred(api_key=FRED_API_KEY)
    print("Successfully connected to FRED")
except Exception as e:
    print("Error connecting to FRED:", e)
    fred = None


def download_fred_series(series_id, start=START_DATE, end=TODAY):
    # Note: This function downloads ONE macro time series from FRED, example series_id values would be like "CPIAUCSL", "UNRATE", etc

    # Note: This print is to make sure the client was created properly, to make sure if problems happen we can find out where what went wrong
    if fred is None:
        print(f"FRED client is not initialized.")
        return None

    print(f"[macro] Downloading FRED series {series_id} from {start} to {end} ...")

    try:
        series = fred.get_series(series_id, observation_start=start, observation_end=end)
    except Exception as e:
        print(f"Error downloading series {series_id}:", e)
        return None

    if series is None or series.empty:
        print(f"Data was not returned for series {series_id}")
        return None

    # Note: Turning the series into a DataFrame with columns: date, value
    df = series.to_frame(name="value")
    df.index.name = "date"
    df = df.reset_index()

    return df


def save_macro(df, name):
    # Note: This function saves the macro DataFrame into data/raw/macro/
    if df is None or df.empty:
        print(f"Nothing to save for {name}")
        return

    out_path = os.path.join(RAW_MACRO_DIR, f"{name}_fred_raw.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved {name} data to {out_path} (rows: {len(df)})")


# Note: Now we loop over all the FRED series we defined earlier in FRED_SERIES
# Note: Keys are our friendly names (cpi, unemployment, etc.), values are the actual FRED IDs
for name, series_id in FRED_SERIES.items():
    df_macro = download_fred_series(series_id)
    save_macro(df_macro, name)


Successfully connected to FRED
[macro] Downloading FRED series CPIAUCSL from 1980-01-01 to 2025-11-12 ...
Saved cpi data to /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/raw/macro/cpi_fred_raw.csv (rows: 549)
[macro] Downloading FRED series UNRATE from 1980-01-01 to 2025-11-12 ...
Saved unemployment data to /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/raw/macro/unemployment_fred_raw.csv (rows: 548)
[macro] Downloading FRED series FEDFUNDS from 1980-01-01 to 2025-11-12 ...
Saved fed_funds_rate data to /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Project/project_code/data/raw/macro/fed_funds_rate_fred_raw.csv (rows: 550)
[macro] Downloading FRED series M2SL from 1980-01-01 to 2025-11-12 ...
Saved m2_money_stock data to /Users/aaryabratc/Desktop/WASHU MAIN/FALL 2025/cse3104_Data_Manipulation_and_Management/Proje

# Google Trends Sentiment Data

In [45]:
# Note: Now we move on to Google Trends sentiment data using pytrends
# Note: Google Trends only goes back to 2004, which is fine
# Note: We'll pull sentiment for these keywords: ["recession", "stock market crash", "bear market", "financial crisis"]

# Note: Okay so first try got code 429 error, indicating that Google did not allow too many requests in a short span
# Note: Basically what happened was it downloaded for some keywords and did not for some, indicating to many requests too quick basically
# Note: Idea is to retry with a waittime basically
# Note: Here's some notes, seems like it was not related to the time issue, since it still is not working with a workaround for 60s waittime
# Note: Upon further research it seems like this has more to do with automated requests
# Note: New idea is to now try one keyword at a time, with a fresh pytrends session for each call and manually control it

# Note: Initialize the pytrends connection
# Note: Can use any timezon but we'll use US for simplicity
# pytrends = TrendReq(hl='en-US', tz=360)


def download_trends_keyword(keyword, start_date=TRENDS_START_DATE, end_date=TODAY):
    # Note: This function downloads the Google Trends "interest over time" for ONE keyword
    # Note: Google Trends needs the format "YYYY-MM-DD YYYY-MM-DD", all in one string

    print(f"Downloading Google Trends data for '{keyword}'...")

    try:
        # Note: Build the timeframe in the format Google Trends expects
        timeframe = f"{start_date} {end_date}"

        # Note: IMPORTANT CHANGE:
        # Note: Instead of one global pytrends object, we create a fresh TrendReq() per keyword call.
        # Note: This sometimes helps when Google has rate-limited a previous session.
        local_pytrends = TrendReq(hl='en-US', tz=360)

        # Note: pytrends requires that we "build a payload" before fetching data
        local_pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo='', gprop='')

        # Note: Now we can request the interest-over-time data.
        df = local_pytrends.interest_over_time()

        if df.empty:
            print(f"No data returned for keyword '{keyword}'")
            return None

        # Note: interest_over_time() returns a dataframe with the keyword and an 'isPartial' column
        # Note: We only care about the keyword and the date, so we will clean it up a bit
        df = df.reset_index()

        # Note: Some column names may have spaces or weird formatting, so we can clean that up too
        df.columns = [c.lower().replace(" ", "_") for c in df.columns]

        # Note: Will also drop the 'ispartial' column since we don’t need it
        if "ispartial" in df.columns:
            df = df.drop(columns=["ispartial"])

        # Note: Also rename the keyword column to something predictable
        df = df.rename(columns={keyword.lower().replace(" ", "_"): "value"})

        return df

    except Exception as e:
        print(f"Error downloading keyword '{keyword}':", e)
        return None


def save_trends(df, keyword):
    # Note: This function saves the sentiment dataframe into data/raw/sentiment/
    if df is None or df.empty:
        print(f"[sentiment] Nothing to save for '{keyword}'")
        return

    # Note: Just making sure to format the filename nicely, like: recession_trends_raw.csv
    filename = f"{keyword.lower().replace(' ', '_')}_trends_raw.csv"
    out_path = os.path.join(RAW_SENTIMENT_DIR, filename)

    df.to_csv(out_path, index=False)
    print(f"Saved keyword '{keyword}' to {out_path} (rows: {len(df)})")


# Note: IMPORTANT CHANGE:
# Note: We are NOT looping through all TRENDS_KEYWORDS automatically anymore.
# Note: Instead, we will call download_trends_keyword() and save_trends() manually, one keyword at a time in separate cells.

In [49]:
# Note: Now going to call it with one keyword at a time to see if that works
# Note: Trying with "recession"

# keyword = "recession"
# df_trend = download_trends_keyword(keyword)
# save_trends(df_trend, keyword)

# Note: Funnily, tried wustl-encrypted, then my personal hotspot, and it did not but worked on eduroam
# Note: Now this kernel will just go through the trendkeywords from the first kernel and download for each keyword
import time

for keyword in TRENDS_KEYWORDS:
    print(f"Starting download for keyword: '{keyword}'")

    df_trend = download_trends_keyword(keyword)
    save_trends(df_trend, keyword)

    # Note: Making sure to have a quick pause so Google does not stop us again, but this should not be an issue since we are starting a new session each time
    time.sleep(30)



Starting download for keyword: 'bear market'
Error downloading keyword 'bear market': The request failed: Google returned a response with code 429
[sentiment] Nothing to save for 'bear market'


KeyboardInterrupt: 